In [ ]:
!pip install transformers torch accelerate yfinance gradio -q
!pip install datasets sentencepiece protobuf plotly -q

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    BartForConditionalGeneration,
    pipeline
)
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import gradio as gr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

class AdvancedFinancialAI:
    def __init__(self):
        print("Initializing AI")

        self.sentiment_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
        self.sentiment_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
        self.sentiment_pipeline = pipeline("sentiment-analysis", model=self.sentiment_model, tokenizer=self.sentiment_tokenizer)

        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

        self.text_gen_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
        self.text_gen_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

        self.zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

        print("AI Ready!")

    def analyze_sentiment_batch(self, texts):
        results = []
        for text in texts:
            try:
                chunks = [text[i:i+512] for i in range(0, len(text), 512)]
                sentiments = [self.sentiment_pipeline(chunk)[0] for chunk in chunks]
                avg_score = np.mean([s['score'] if s['label'] == 'positive' else -s['score'] if s['label'] == 'negative' else 0 for s in sentiments])

                if avg_score > 0.3:
                    label = 'positive'
                elif avg_score < -0.3:
                    label = 'negative'
                else:
                    label = 'neutral'

                results.append({
                    'text': text[:100] + '...' if len(text) > 100 else text,
                    'label': label,
                    'score': abs(avg_score),
                    'confidence': self._get_confidence(abs(avg_score))
                })
            except:
                results.append({'text': text[:100], 'label': 'error', 'score': 0, 'confidence': 'N/A'})
        return results

    def _get_confidence(self, score):
        if score > 0.75: return 'Very High'
        elif score > 0.5: return 'High'
        elif score > 0.3: return 'Medium'
        return 'Low'

    def get_stock_data(self, ticker, period="6mo"):
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period=period)
            info = stock.info

            news = stock.news[:5] if hasattr(stock, 'news') else []

            return {
                'history': hist,
                'current_price': info.get('currentPrice', hist['Close'][-1] if len(hist) > 0 else 0),
                'market_cap': info.get('marketCap', 'N/A'),
                'pe_ratio': info.get('trailingPE', 'N/A'),
                'company_name': info.get('longName', ticker),
                'sector': info.get('sector', 'N/A'),
                'beta': info.get('beta', 'N/A'),
                'dividend_yield': info.get('dividendYield', 0),
                'news': news,
                'description': info.get('longBusinessSummary', '')
            }
        except Exception as e:
            return {'error': str(e)}

    def calculate_advanced_indicators(self, df):
        df['SMA_20'] = df['Close'].rolling(window=20).mean()
        df['SMA_50'] = df['Close'].rolling(window=50).mean()
        df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
        df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = df['EMA_12'] - df['EMA_26']
        df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))

        df['BB_middle'] = df['Close'].rolling(window=20).mean()
        bb_std = df['Close'].rolling(window=20).std()
        df['BB_upper'] = df['BB_middle'] + (bb_std * 2)
        df['BB_lower'] = df['BB_middle'] - (bb_std * 2)

        df['Volatility'] = df['Close'].pct_change().rolling(window=20).std() * np.sqrt(252) * 100

        df['Returns'] = df['Close'].pct_change()

        return df

    def generate_ai_report(self, ticker, data, signals):
        prompt = f"""Financial Analysis Report for {data['company_name']} ({ticker}):

Current Price: ${signals['current_price']}
Sector: {data['sector']}
Market Sentiment: {signals['market_sentiment']}
Technical Signal: {signals['technical_signal']}

Key Indicators:
- RSI: {signals['rsi']}
- MACD: {signals['macd_signal']}
- Volatility: {signals['volatility']:.2f}%

Investment Analysis:"""

        inputs = self.text_gen_tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

        with torch.no_grad():
            outputs = self.text_gen_model.generate(
                inputs.input_ids,
                max_length=400,
                num_return_sequences=1,
                temperature=0.8,
                do_sample=True,
                top_p=0.92,
                top_k=50,
                repetition_penalty=1.2,
                pad_token_id=self.text_gen_tokenizer.eos_token_id
            )

        return self.text_gen_tokenizer.decode(outputs[0], skip_special_tokens=True)

    def analyze_news_sentiment(self, news_list):
        if not news_list:
            return "No recent news available"

        sentiments = []
        for item in news_list:
            title = item.get('title', '')
            if title:
                sent = self.sentiment_pipeline(title[:512])[0]
                sentiments.append(sent)

        if sentiments:
            positive = sum(1 for s in sentiments if s['label'] == 'positive')
            negative = sum(1 for s in sentiments if s['label'] == 'negative')
            neutral = len(sentiments) - positive - negative

            if positive > negative:
                return f"📈 Bullish ({positive} positive, {negative} negative, {neutral} neutral)"
            elif negative > positive:
                return f"📉 Bearish ({positive} positive, {negative} negative, {neutral} neutral)"
            else:
                return f"➡️ Neutral ({positive} positive, {negative} negative, {neutral} neutral)"

        return "Insufficient data"

    def generate_comprehensive_analysis(self, ticker):
        ticker = ticker.upper().strip()

        data = self.get_stock_data(ticker, period="6mo")
        if 'error' in data:
            return f"❌ Error: {data['error']}", None, None

        df = data['history'].copy()
        if len(df) == 0:
            return "❌ No historical data available", None, None

        df = self.calculate_advanced_indicators(df)

        latest = df.iloc[-1]
        prev = df.iloc[-5] if len(df) > 5 else df.iloc[0]

        signals = {
            'ticker': ticker,
            'company': data['company_name'],
            'current_price': round(latest['Close'], 2),
            'price_change': round(((latest['Close'] - prev['Close']) / prev['Close']) * 100, 2),
            'rsi': round(latest['RSI'], 2),
            'macd': round(latest['MACD'], 4),
            'macd_signal': 'Bullish' if latest['MACD'] > latest['Signal_Line'] else 'Bearish',
            'volatility': round(latest['Volatility'], 2),
            'volume': int(latest['Volume']),
            'market_cap': data['market_cap'],
            'pe_ratio': data['pe_ratio'],
            'sector': data['sector'],
            'beta': data['beta']
        }

        signals['market_sentiment'] = self.analyze_news_sentiment(data['news'])

        technical_signals = []
        score = 0

        if latest['Close'] > latest['SMA_20'] > latest['SMA_50']:
            technical_signals.append("🟢 Strong uptrend - Price above both moving averages")
            score += 2
        elif latest['Close'] < latest['SMA_20'] < latest['SMA_50']:
            technical_signals.append("🔴 Strong downtrend - Price below both moving averages")
            score -= 2

        if latest['RSI'] < 30:
            technical_signals.append("🟢 Oversold (RSI < 30) - Potential buying opportunity")
            score += 1
        elif latest['RSI'] > 70:
            technical_signals.append("🔴 Overbought (RSI > 70) - Potential correction ahead")
            score -= 1

        if latest['MACD'] > latest['Signal_Line']:
            technical_signals.append("🟢 MACD Bullish crossover")
            score += 1
        else:
            technical_signals.append("🔴 MACD Bearish crossover")
            score -= 1

        if latest['Close'] < latest['BB_lower']:
            technical_signals.append("🟢 Price below lower Bollinger Band - Oversold")
            score += 1
        elif latest['Close'] > latest['BB_upper']:
            technical_signals.append("🔴 Price above upper Bollinger Band - Overbought")
            score -= 1

        if signals['price_change'] > 5:
            technical_signals.append(f"🟢 Strong momentum: +{signals['price_change']}%")
            score += 1
        elif signals['price_change'] < -5:
            technical_signals.append(f"🔴 Weak momentum: {signals['price_change']}%")
            score -= 1

        if score >= 3:
            recommendation = "🟢 STRONG BUY"
            rec_color = "#00ff00"
        elif score >= 1:
            recommendation = "🟡 BUY"
            rec_color = "#90EE90"
        elif score <= -3:
            recommendation = "🔴 STRONG SELL"
            rec_color = "#ff0000"
        elif score <= -1:
            recommendation = "🟠 SELL"
            rec_color = "#FFA500"
        else:
            recommendation = "⚪ HOLD"
            rec_color = "#808080"

        signals['technical_signal'] = recommendation
        signals['signals'] = technical_signals
        signals['score'] = score

        report = f"""
AI Analysis Report

{signals['company']} ({ticker})
Sector:{signals['sector']} | Current Price: ${signals['current_price']} ({'+' if signals['price_change'] > 0 else ''}{signals['price_change']}%)

---

AI Recommendation: {recommendation}
Confidence Score: {abs(score)}/5**

---

 📈 Technical Indicators

| Indicator | Value | Status |
|-----------|-------|--------|
| RSI | {signals['rsi']} | {'Oversold' if signals['rsi'] < 30 else 'Overbought' if signals['rsi'] > 70 else 'Neutral'} |
| MACD | {signals['macd']} | {signals['macd_signal']} |
| Volatility | {signals['volatility']}% | {'High' if signals['volatility'] > 30 else 'Medium' if signals['volatility'] > 20 else 'Low'} |
| Beta | {signals['beta']} | {'High Risk' if isinstance(signals['beta'], (int, float)) and signals['beta'] > 1.5 else 'Moderate' if isinstance(signals['beta'], (int, float)) and signals['beta'] > 1 else 'Low Risk'} |
| P/E Ratio | {signals['pe_ratio']} | {'Overvalued' if isinstance(signals['pe_ratio'], (int, float)) and signals['pe_ratio'] > 30 else 'Fair' if isinstance(signals['pe_ratio'], (int, float)) and signals['pe_ratio'] > 15 else 'Undervalued'} |

---

🔍 Technical Signals Detected

"""
        for signal in technical_signals:
            report += f"- {signal}\n"

        report += f"\n---\n\n## 📰 Market Sentiment\n{signals['market_sentiment']}\n\n---\n\n"

        ai_insight = self.generate_ai_report(ticker, data, signals)
        report += f"AI-Generated Insights\n\n{ai_insight}\n\n---\n\n"



        chart = self.create_interactive_chart(df, signals)

        return report, chart, data.get('description', 'No description available')

    def create_interactive_chart(self, df, signals):
        fig = make_subplots(
            rows=3, cols=1,
            shared_xaxis=True,
            vertical_spacing=0.05,
            row_heights=[0.5, 0.25, 0.25],
            subplot_titles=('Price & Technical Indicators', 'MACD', 'RSI')
        )

        fig.add_trace(go.Candlestick(
            x=df.index,
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='Price'
        ), row=1, col=1)

        fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], name='SMA 20', line=dict(color='orange', width=1)), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], name='SMA 50', line=dict(color='blue', width=1)), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df['BB_upper'], name='BB Upper', line=dict(color='gray', width=1, dash='dash')), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df['BB_lower'], name='BB Lower', line=dict(color='gray', width=1, dash='dash')), row=1, col=1)

        fig.add_trace(go.Scatter(x=df.index, y=df['MACD'], name='MACD', line=dict(color='blue', width=2)), row=2, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=df['Signal_Line'], name='Signal', line=dict(color='red', width=2)), row=2, col=1)

        fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], name='RSI', line=dict(color='purple', width=2)), row=3, col=1)
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)

        fig.update_layout(
            height=900,
            showlegend=True,
            xaxis_rangeslider_visible=False,
            template='plotly_dark'
        )

        fig.update_yaxes(title_text="Price ($)", row=1, col=1)
        fig.update_yaxes(title_text="MACD", row=2, col=1)
        fig.update_yaxes(title_text="RSI", row=3, col=1)

        return fig

    def batch_sentiment_analysis(self, text_input):
        texts = [t.strip() for t in text_input.split('\n') if t.strip()]

        if not texts:
            return "Please enter at least one text to analyze"

        results = self.analyze_sentiment_batch(texts)

        output = "# 📊 Sentiment Analysis Results\n\n"

        for i, result in enumerate(results, 1):
            emoji = "🟢" if result['label'] == 'positive' else "🔴" if result['label'] == 'negative' else "⚪"
            output += f"## {emoji} Text {i}\n"
            output += f"**Text:** {result['text']}\n\n"
            output += f"**Sentiment:** {result['label'].upper()}\n\n"
            output += f"**Confidence:** {result['confidence']} ({result['score']:.2%})\n\n"
            output += "---\n\n"

        positive = sum(1 for r in results if r['label'] == 'positive')
        negative = sum(1 for r in results if r['label'] == 'negative')
        neutral = len(results) - positive - negative

        output += f"\n### Summary: {positive} Positive | {negative} Negative | {neutral} Neutral\n"

        return output

print("="*70)
print("INITIALIZING ALPHASIGNAL AI")
print("="*70)

analyzer = AdvancedFinancialAI()

with gr.Blocks(theme=gr.themes.Soft(), title="AlphaSignal AI") as demo:
    gr.Markdown("""
    AI
    Advanced Financial Intelligence Platform powered by Hugging Face

    Real-time stock analysis with AI-powered insights, technical indicators, and sentiment analysis.
    """)

    with gr.Tabs():
        with gr.Tab("📈 Stock Analysis"):
            with gr.Row():
                with gr.Column(scale=1):
                    ticker_input = gr.Textbox(
                        label="Stock Ticker Symbol",
                        placeholder="Enter ticker (e.g., AAPL, MSFT, TSLA)",
                        value="AAPL"
                    )
                    analyze_btn = gr.Button("🔍 Analyze Stock", variant="primary", size="lg")

                    gr.Markdown("""
                    **Popular Tickers:**
                    - Tech: AAPL, MSFT, GOOGL, NVDA, TSLA, META
                    - Finance: JPM, BAC, GS, V, MA
                    - Retail: AMZN, WMT, COST
                    """)

                with gr.Column(scale=2):
                    report_output = gr.Markdown(label="Analysis Report")

            with gr.Row():
                chart_output = gr.Plot(label="Interactive Technical Chart")

            with gr.Row():
                company_info = gr.Textbox(label="Company Description", lines=5)

            analyze_btn.click(
                fn=analyzer.generate_comprehensive_analysis,
                inputs=[ticker_input],
                outputs=[report_output, chart_output, company_info]
            )

        with gr.Tab("💬 Sentiment Analysis"):
            gr.Markdown("""
            ### Analyze Financial Text Sentiment
            Enter financial news, earnings reports, or market commentary (one per line)
            """)

            with gr.Row():
                with gr.Column():
                    sentiment_input = gr.Textbox(
                        label="Financial Texts",
                        placeholder="Enter text to analyze (one per line)...\n\nExample:\nApple reports record earnings beating expectations\nThe company faces regulatory challenges",
                        lines=10
                    )
                    sentiment_btn = gr.Button("🔍 Analyze Sentiment", variant="primary")

                with gr.Column():
                    sentiment_output = gr.Markdown(label="Sentiment Results")

            gr.Examples(
                examples=[
                    ["Apple reported record Q4 earnings, exceeding analyst expectations with strong iPhone sales\nThe Federal Reserve signals potential interest rate cuts amid economic concerns\nTesla stock surges on better-than-expected delivery numbers"],
                    ["Company faces significant regulatory challenges in European markets\nQuarterly revenue misses expectations due to supply chain disruptions\nCEO announces major restructuring plan to cut costs"],
                ],
                inputs=sentiment_input
            )

            sentiment_btn.click(
                fn=analyzer.batch_sentiment_analysis,
                inputs=[sentiment_input],
                outputs=[sentiment_output]
            )


demo.launch(share=True, debug=True)

INITIALIZING ALPHASIGNAL AI
Initializing AI


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


AI Ready!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8f990526f0eb429f82.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
